# Age and gender classification with CNN
Tento dokument je výsledkom práce študentov Tomáša Mifkoviča a Benjamína Jozefa Šiškoviča ako návrh projektu na predmet 
Neurónové siete na FIIT STU v aka. roku 2019/2020.  
  
## Motivácia  
Ako projekt sme si vybrali tému **Určenie veku na základe fotografie**. Táto téma je zaujímavá vo viacerých smeroch, 
pretože by sa dala ďalej rozvýjať, hlavne ako zabezpečenie napríklad mobilných zariadení. V našom prípade by sa skôr 
mohla použiť na rozpoznanie veku používateľa live v aplikáciach na mobilných zariadeniach alebo počítačoch.  
  
Taktiež, v čase keď sa na internete mnohokrát zverejňujú detské fotky, ktoré sa dajú ľahko zneužiť, alebo môžu vystaviť 
deti do nebezpečných či nepríjemných situácií. Rozpoznanie tváre by mohlo znížíť počet takýchto fotografií, keď pri 
uploade fotiek vyskočí okno s upozornením.  
  
Náš predmet záujmu spočíva hlavne v tom, aby sme vedeli fotky čo najpresnejšie zaradiť do vekových kategórií deti, 
mladistvý, mladý, dospelý, dôchodca. Presnejšie, budeme ich zaraďovať do vekových kategórii (0-4), (5-9), (10-17), 
(18-25), (26-39),(40-59),(60-74),(75-).
  
## Podobné práce  
V dnešnom modernom svete existuje veľa rozličných riešení na daný problém. Väčšina veľkých celosvetových firiem si 
buduje vlastné softvéry alebo služby, ktoré implementujú určitú formu umelej inteligencie (presnejšie neurónovú sieť) 
na rozpoznanie tváre tzv. face recognition, ktorá sa neustále učí používaním API. Každá takáto spoločnosť si model 
buduje, trénuje a testuje sama (väčšinou bez zverejnenia svojeho know-how). Medzi takéto firmy môžme zaradiť Apple, 
Microsoft, Google, či iné. Ako najznámejší príklad na rozpoznanie tváre môžme použiť FaceID od spoločnosti Apple.

**How-old.net** - webová stránka od spoločnosti Microsoft, ktorá dokáže z fotky identifikovať tvár človeka a snaží sa 
uhádnuť pohlavie a vek osoby na fotke

**aws.amazon.com/rekognition** - je projekt spoločnosti Amazon, ktorý dokáže rozpoznávať z videí a fotiek rôzne objekty, 
ale predovšetkým aj tváre ľudí. Aplikácia dokáže zistiť pohlavie človeka, či sa usmieva, či je šťastný či má okuliare a pod.

**https://www.learnopencv.com/age-gender-classification-using-opencv-deep-learning-c-python/?fbclid=IwAR0rxSPssBGnjFOm3WsMx3jp7hqA24hicUu4iFADi9DrNjj0qQaALCERGuU** - je open-source riešenie na tento problém. V tomto riešení používajú konvolučnú neurónovú sieť (3 konvolučné vrstvy, 2 plne prepojené vrstvy a 1 výstupnú vrstvu). Neurónová sieť odhaduje vek a pohlavie osoby na fotke. Vek odhaduje klasifikačným prístupom - čiže snaží sa odhadnúť vekovú skupinu ľudí namiesto presného veku. V takomto prípade sú výsledky presnejšie a oveľa jednoduchšie vypočítať výsledok.
## Databázy vhodné na použitie
Aby sme vedeli náš model čo najlepšie vytrénovať a kvalitne ho otestovať, potrebujeme veľké množstvo fotiek. Dolu 
vypísané webové linky nám poskytujú voľne dostupný zdroj fotiek osôb spolu s dodatočnými informáciami ako pohlavie či 
vek osoby na fotke.

[1] **https://talhassner.github.io/home/projects/Adience/Adience-data.html** - The Open University of Israel poskytujú 
databázu nefiltrovaných snímok tvári ľudí na klasifikáciu pohlavia a veku človeka (celkovo 26 580 snímok) s 8 vekovými 
skupinami (0-2, 4-6, 8-13, 15-20, 25-32, 38-43, 48-53, 60-)

[2] **https://github.com/JingchunCheng/All-Age-Faces-Dataset?fbclid=IwAR0xZCjKKWhJBbce__VZdTO4SN62hbn_45pbB-h6AgO-gDboi_d9vNjOqQY** - voľne dostupný zdroj fotiek tvári ľudí prevažne z ázie. Celkový počet je 13 322 fotiek s vekovým rozpätím od 2 do 80 rokov.

[3] **https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/?fbclid=IwAR07Vw9R8yi2P_D5EiXBCfZ3CYeNo92b5Z9ZWkuHo_tnWoq_8Aw5TRvhxWQ** - voľne dostupná databáza fotiek z celého sveta, ktoré sú vhodné na trénovanie a testovanie neurónových sietí. Celkový počet je viac ako 500 000.

[4] **https://susanqq.github.io/UTKFace/?fbclid=IwAR2IuscY8oJ6ubuUdKCMRgCtZ0ZQJ38VMMykgbwK7WZK3LLvszBGp26HOyY** - 
voľne dostupná databáza fotiek poskytujúca doplňujúce informácie o fotkách s celkovým počtom viac ako 20 000.

## High-level solution proposal 
V čase písania návrhu sme absolvovali prednášku z predmetu o téme zisťovania hrán v obrázkoch. Túto tému si budeme 
musieť naštudovať podrobnejšie.  
Po natrénovaní neurónovej siete na datasetoch spomenutých vyššie vyskúšame fotky  
  
- naše, teda presne vieme, aký vek sme na nich mali  
- vybraté fotky z datasetov, pre overenie, či je to priamo naučené alebo nastala nejaká odchylka pomocou confusion matrix
  
Údaje, ktoré nám poskytne neurónová sieť porovnáme so skutočnými hodnotami a tak vypočítame odchylku hodnôt. Budeme sa 
zároveň sústrediť, v ktorom veku nastala najväčšia odchylka a zistíme, prečo sa tak stalo.

# Použité databázy

Ako prvotné testovacie dáta sme použili voľne dostupný dataset [2].
Dataset obsahuje 13 322 fotiek z toho 7 381 je žien a 5 941 mužov. Vekové rozpätie je od 2 do 80 rokov. 
Dataset poskytuje aj upravené/zarovnané/otočené fotky, aby bolo lepšie vidieť do tváre. Pri testovaní, 
či je lepšie používať originály fotky alebo upravená verzia vyšli lepšie výsledky pre originály 
v porovnaní dosiahnutej presnosti. Dáta sú vo formáte XXXXXAYY.jpg, kde XXXXX je poradové číslo a ked je menšie 
ako 7 382 tak sa jedná o ženu, a keď je väčšie rovné tak je to muž. YY je vek človeka na fotke.

Ako druhý dataset sme použili [4]. Dataset poskytuje 23 708 fotiek ľudí od veku 1 do 116. Dáta sú v tvare Y_X_Z.jpg, 
Y je vek človeka na fotke, X je z množiny {0, 1} hovoriac pohlavie a Z je identifikačné číslo.

Ako tretí dataset sme použili [1]. Dataset obsahuje 26 580 fotiek, pričom niektoré neboli použiteľné. Dáta sú v 
tvare Y_X_Z.jpg, Y je vek človeka na fotke, X je z množiny {0, 1} hovoriac pohlavie a Z je identifikačné číslo.

Celkovo teda máme dataset fotiek o veľkosti ‭54 510‬ fotiek. Tento dataset sme rozdelili na 4/5 trénovaciu jednotku (43 608)
 a 1/5 na testovaciu (10 902).

Predtým ako sa fotky pošlu do siete sa musia trochu upraviť. Fotky sú upravované funkciami opencv napr. resize() 
na rovnakú veľkosť (konkrétne 64x64) a delené rgb kanály hodnotou 255, aby sa hodnoty normalizovali. 
Vytvorili sme vekové kategórie (0-4),(5-9),(10-17),(18-25),(26-39),(40-59),(60-74),(75-), ktoré reprezentujú čísla 
od 0-7 (8 kategórii).

<img src="src/age_analysis.png">

<img src="src/gender_analysis.png">

In [ ]:
import glob
from os.path import basename
import numpy as np
from tqdm import tqdm
import random as ra
from cv2 import *

MUZ = 0
ZENA = 1
HEIGHT = 64
WIDTH = 64
DIM = (HEIGHT, WIDTH)


def prep_img(file):
    image = imread(file)
    image = resize(image, DIM)
    image = image / 255.0
    image = np.array(image)
    return image


def decide_age(age):
    age = int(age)
    if age < 5:
        return 0
    if age < 10:
        return 1
    if age < 18:
        return 2
    if age < 26:
        return 3
    if age < 40:
        return 4
    if age < 60:
        return 5
    if age < 75:
        return 6
    return 7


def decide_gender(number):
    number = int(number)
    if number < 7381:
        return ZENA
    return MUZ


def get_gender(file):
    if file[1] == '_':
        gender = int(file[2])
    elif file[2] == '_':
        gender = int(file[3])
    else:
        gender = int(file[4])
    return gender


def get_age(file):
    if file[1] == '_':
        age = int(file[0])
    elif file[2] == '_':
        age = int(file[0] + file[1])
    else:
        age = int(file[0] + file[1] + file[2])
    return decide_age(age=age)


def prepare_data(images, age_labels, gender_labels):
    train_images = []
    train_age_labels = []
    train_gender_labels = []
    test_images = []
    test_age_labels = []
    test_gender_labels = []

    pom = list(zip(images, age_labels, gender_labels))
    ra.shuffle(pom)
    images, age_labels, gender_labels = zip(*pom)

    for x in tqdm(range(len(images))):
        if x < int(len(images) * 4 / 5):
            train_images.append(images[x])
            train_age_labels.append(age_labels[x])
            train_gender_labels.append(gender_labels[x])
        else:
            test_images.append(images[x])
            test_age_labels.append(age_labels[x])
            test_gender_labels.append(gender_labels[x])

    train_images = np.array(train_images)
    train_age_labels = np.array(train_age_labels)
    train_gender_labels = np.array(train_gender_labels)
    test_images = np.array(test_images)
    test_age_labels = np.array(test_age_labels)
    test_gender_labels = np.array(test_gender_labels)

    return train_images, train_age_labels, train_gender_labels, test_images, test_age_labels, test_gender_labels


def process_dataset3():
    images = []
    age_labels = []
    gender_labels = []

    for file in tqdm(glob.glob("..\\photos\\dataset3\\*.jpg")[:50]):
        filename = basename(file)
        images.append(prep_img(file=file))
        age_labels.append(get_age(file=filename))
        gender_labels.append(get_gender(file=filename))

    return images, age_labels, gender_labels


def process_dataset2():
    images = []
    age_labels = []
    gender_labels = []

    for file in tqdm(glob.glob("..\\photos\\dataset2\\*.jpg")[:50]):
        filename = basename(file)
        images.append(prep_img(file=file))
        age_labels.append(get_age(file=filename))
        gender_labels.append(get_gender(file=filename))

    return images, age_labels, gender_labels


def process_dataset1():
    images = []
    age_labels = []
    gender_labels = []

    for file in tqdm(glob.glob("..\\photos\\dataset1\\*.jpg")[:50]):
        filename = basename(file)
        images.append(prep_img(file=file))
        age_labels.append(decide_age(age=filename[6:8]))
        gender_labels.append(decide_gender(number=filename[0:5]))

    return images, age_labels, gender_labels


def get_all_data():
    print("Preparing dataset...")
    images1, age_labels1, gender_labels1 = process_dataset1()
    images2, age_labels2, gender_labels2 = process_dataset2()
    images3, age_labels3, gender_labels3 = process_dataset3()

    images = np.concatenate((images1, images2, images3))
    age_labels = np.concatenate((age_labels1, age_labels2, age_labels3))
    gender_labels = np.concatenate((gender_labels1, gender_labels2, gender_labels3))

    return images, age_labels, gender_labels


def get_data():
    print("Preparing dataset...")
    images1, age_labels1, gender_labels1 = process_dataset1()
    images2, age_labels2, gender_labels2 = process_dataset2()
    images3, age_labels3, gender_labels3 = process_dataset3()

    images = np.concatenate((images1, images2, images3))
    age_labels = np.concatenate((age_labels1, age_labels2, age_labels3))
    gender_labels = np.concatenate((gender_labels1, gender_labels2, gender_labels3))

    print("Preparing train and test data...")
    return prepare_data(images, age_labels, gender_labels)

# Model

Finálny model je zmenšená verzia modelu VGG16, ktorý je často používaný ako základ pre CNN modely. Pozostáva zo 4 skupín
konvolučných vrstiev. Každá skupina je doplnená vrstvou BatchNormalizetion a MaxPooling vrstovou na zlepšenie trénovania.
Prvá skupina má 32 vrchov v konvolučnej vrstve, druhá 64, tretia 128 a posledná 256. Potom sme model rozvetvili na 2 
vetvy, pretože máme multioutput model. Každá vetva je potom ešte doplnená 2 Dense vrstvami o veľkosti 256 a Dropout
vrstvami na zlepšenie trénovanie, aby sme trochu obmedzili overfitting modelu. Taktiež používame L2 regularizátor na
zlepšenie trénovania a obmedzeniu pre-trénovania.

<img src="src/model.png">

Layer (type)                    Output Shape         Param #     Connected to                     
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 32)   9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 32)   128         conv2d_2[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 32, 32, 64)   18496       max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 32, 32, 64)   36928       conv2d_3[0][0]                   
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 32, 32, 64)   256         conv2d_4[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 16, 16, 64)   0           batch_normalization_2[0][0]      
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 16, 16, 128)  73856       max_pooling2d_2[0][0]            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 16, 16, 128)  147584      conv2d_5[0][0]                   
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 16, 16, 128)  512         conv2d_6[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 8, 8, 128)    0           batch_normalization_3[0][0]      
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 8, 8, 256)    295168      max_pooling2d_3[0][0]            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 8, 8, 256)    590080      conv2d_7[0][0]                   
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 8, 8, 256)    1024        conv2d_8[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 4, 4, 256)    0           batch_normalization_4[0][0]      
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 4096)         0           max_pooling2d_4[0][0]            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 256)          1048832     flatten_1[0][0]                  
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 256)          1048832     flatten_1[0][0]                  
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 256)          0           dense_1[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 256)          0           dense_3[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 256)          65792       dropout_1[0][0]                  
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 256)          65792       dropout_3[0][0]                  
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 256)          0           dense_2[0][0]                    
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 256)          0           dense_4[0][0]                    
__________________________________________________________________________________________________
age (Dense)                     (None, 8)            2056        dropout_2[0][0]                  
__________________________________________________________________________________________________
gender (Dense)                  (None, 1)            257         dropout_4[0][0]                  
__________________________________________________________________________________________________
Total params: 3,405,737
Trainable params: 3,404,777
Non-trainable params: 960

In [ ]:
from keras.models import Model
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Input, Dropout, BatchNormalization
from keras.regularizers import l2

def create_model(height, width, pocet_vekovych_kategorii):
    input_layer = Input(shape=(width, height, 3))

    con1 = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(input_layer)
    con11 = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(con1)
    batch1 = BatchNormalization()(con11)
    max1 = MaxPooling2D(pool_size=(2, 2))(batch1)

    con2 = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(max1)
    con22 = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(con2)
    batch2 = BatchNormalization()(con22)
    max2 = MaxPooling2D(pool_size=(2, 2))(batch2)

    con3 = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(max2)
    con33 = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(con3)
    batch3 = BatchNormalization()(con33)
    max3 = MaxPooling2D(pool_size=(2, 2))(batch3)

    con4 = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(max3)
    con44 = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(con4)
    batch4 = BatchNormalization()(con44)
    max4 = MaxPooling2D(pool_size=(2, 2))(batch4)

    flat = Flatten()(max4)

    # age
    age_dense1 = Dense(units=256, kernel_regularizer=l2(l=0.005), activation='relu')(flat)
    age_drop1 = Dropout(rate=0.5)(age_dense1)
    age_dense2 = Dense(units=256, kernel_regularizer=l2(l=0.005), activation='relu')(age_drop1)
    age_drop2 = Dropout(rate=0.5)(age_dense2)
    age = Dense(units=pocet_vekovych_kategorii, activation='softmax', name="age")(age_drop2)

    # gender
    gender_dense1 = Dense(units=256, kernel_regularizer=l2(0.025), activation='relu')(flat)
    gender_drop1 = Dropout(rate=0.5)(gender_dense1)
    gender_dense2 = Dense(units=256, kernel_regularizer=l2(0.025), activation='relu')(gender_drop1)
    gender_drop2 = Dropout(rate=0.5)(gender_dense2)
    gender = Dense(units=1, activation='sigmoid', name="gender")(gender_drop2)

    model = Model(inputs=input_layer, outputs=[age, gender])

    return model


class MyNeuralNetwork:
    def __init__(self):
        super(MyNeuralNetwork, self).__init__(name='MyNeuralNetwork')


# Trénovanie

Trénovanie pozostávalo z 30 epoch na dátach rozdelených do 64 velkých baličkov (batches). Určili sme si opitmizer Adam
s learning ratom štandardným 0.001. Určili sme si loss funkcie pre jednotlivé output vrstvy. Pre Age sme vybrali 
sparse categorical crossentropy a pre Gender sme vybrali binary_crossentropy, keďže ide len o clasifikáciu 0 alebo 1. 
Určili sme si metriky - štandardne sme ostali pri presnosti (accuracy) jednotlivých výstupov. Prvé treningy modelu 
boli bez EarlyStoppingu, aby sme videli ako sa náš model správa počas všetkých 30 epochách. Všetky trénovania sme 
logovali prostredníctvom TensorBoardu do logovacích súborov. Trénovanie modelu prebiehalo na rozdelených dátach 
opísaných v kapitole "Použité databázy". Zadali sme hodnotu shuffle=True, aby sme dosiahli náhodné poradie batchov 
pre každú epochu. Po dotrénovaní sme si model s jeho parametrami uložili do súboru "model.h5".

In [ ]:
from keras.callbacks import TensorBoard, EarlyStopping
from datetime import datetime
from src.model import create_model
from keras.optimizers import Adam
from src.import_data import *

BATCH_SIZE = 64
EPOCHS = 30

train_images, train_age_labels, train_gender_labels, test_images, test_age_labels, test_gender_labels = get_data()

model = create_model(HEIGHT, WIDTH, 8)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={"age": "sparse_categorical_crossentropy", "gender": "binary_crossentropy"},
    metrics={"age": "accuracy", "gender": "accuracy"}
)

callbacks = [
    EarlyStopping(monitor='val_loss', mode="min", verbose=1, patience=5),
    TensorBoard(
        log_dir=os.path.join("..\\logs\\", str(datetime.now().strftime("%b_%d_%Y_%H_%M_%S"))),
        histogram_freq=1,
        profile_batch=0)
]

history = model.fit(
    x=train_images,
    y={"age": train_age_labels, "gender": train_gender_labels},
    batch_size=BATCH_SIZE,
    shuffle=True,
    validation_data=(test_images, {"age": test_age_labels, "gender": test_gender_labels}),
    callbacks=callbacks,
    epochs=EPOCHS
)

model.save("model.h5")
model.summary()

# Výsledky
Po každom jednom trénovaní sme si zobrazili pomocou knižnice matlibplot výslekdy testovania. Uložili sme si 4 obrázky, 
konkrétne presnosť veku, presnosť pohlavia, chyby vo veku a chyby v pohlaví. Výsledky sme porovnávali z predošlými 
trénovaniami, aby sme zistili, či sme správne upravovali model a či sa len nezhoršoval.

### Age Accuracy
<img src="src/model_age_accuracy.png">

### Age Loss
<img src="src/model_age_loss.png">

### Gender Accuracy
<img src="src/model_gender_accuracy.png">

### Gender Loss
<img src="src/model_gender_loss.png">

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['age_accuracy'])
plt.plot(history.history['val_age_accuracy'])
plt.title('Model Age Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
plt.savefig('model_age_accuracy.png')
plt.clf()

plt.plot(history.history['gender_accuracy'])
plt.plot(history.history['val_gender_accuracy'])
plt.title('Model Gender Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
plt.savefig('model_gender_accuracy.png')
plt.clf()

plt.plot(history.history['age_loss'])
plt.plot(history.history['val_age_loss'])
plt.title('Model Age Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
plt.savefig('model_age_loss.png')
plt.clf()

plt.plot(history.history['gender_loss'])
plt.plot(history.history['val_gender_loss'])
plt.title('Model Gender Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
plt.savefig('model_gender_loss.png')
plt.clf()

# Experimenty
Náš model pri checkpointe vyzeral nasledovne

In [ ]:
layers = [
   Conv2D(filters=16,kernel_size=7,padding='same',activation='relu'),
   MaxPooling2D(pool_size=(2, 2)),
   Conv2D(filters=32,kernel_size=5,padding='same',activation='relu'),
   MaxPooling2D(pool_size=(2, 2)),
   Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'),
   MaxPooling2D(pool_size=(2, 2)),
   Flatten(),
   Dense(units=512,activation='relu'),
   Dense(units=16,activation='softmax')]
   

Neobsahuje ani multioutput vrstvy a sustreďovali sme sa len na jeden výstup. To sa ukázalo ako veľmi neefektívne rišenie.
Preto sme náš model upravili na multioutput - age, gender. Ako pri prvom checkpointe sme dosahovali príliš vysoký 
overfitting. Po konzultáciách nám bolo odporúčané pridať regularizátory + dropout vrstvy a BatchNormalization vrstvy.

Celkovo sme model zmenili 10 krát, a skúšali ktorý vychádza najlepšie a kde dosahujeme najmenší overfitting. Zmeny boli
vykonané hlavne pridaním/menením optimalizačných aelbo regularizačných parametrov. Najlepšie nám vyšiel model, ktorý je 
opísaný v kapitole "Model". Aj na základe výsledkov finálneho modelu máme stále problém s pretrénovaním modelu po 
približne 15 epoche. Aby sme nemuseli presne zisťovať kedy vypnuť trénovanie a manuálne zadávať počet epoch, použili sme
 EarlyStopping, ktorý sme zamerali na celkovú validačnú loss hodnotu. Hovorí sa, že ak začne stúpať loss, väčšinou to 
 už lepšie nebude, a preto vtedy zastavíme trénovanie a model uložíme s natrénovanými parametrami. Všetky trénovania sú
  zaznamenané v logovacích súboroch. Ak by sme mali ohodnotiť model na jednotlivé výstupy tak, klasifikácia pohlavia 
  dopadla veľmi dobre, na trénovacích dátach sme dosahovali výsledky 97-99% presnosť a na validačných 90-92%. Horšie
  na tom bola časť klasifikácie veku kde sme na trénovacích dátach dosiahli presnosť viac ako 90%, ale na validačných 
  sa to bohužial zastavilo len pri 57-59% - čo je v podstate tiež slušné číslo, keďže sa to kategorizuje na 8 výstupov.
  
### Test modelu
Ako test sme použili naše fotky, ktoré model vôbec nikdy nevidel. Zobrazená tebuľka ukazuje ako nás odhadol.

FOTKA C:  0
Age:  3  Gender:  0
Predicted Age:  4  Gender:  0

FOTKA C:  1
Age:  3  Gender:  0
Predicted Age:  4  Gender:  0

Model správne odhadol naše pohlavie - 0 = 0, sme muži. Ale o 1 kategóriu zle zaradil vek. Patríme do vekovej katégorie 
3 (18-25), ale odhadol nás na vekovú kategóriu 4 (26-39). Keďže sa sekol len o jednu kategóriu nie je to až tak zlé. 
Kľudne nás mohol odhadnúť na 27 ročných mužov.

In [ ]:
from src.import_data import *
from keras.models import load_model


def guess():
    images = []
    age_labels = []
    gender_labels = []

    for file in tqdm(glob.glob("..\\photos\\guess\\*.jpg")):
        filename = basename(file)
        images.append(prep_img(file=file))
        age_labels.append(get_age(file=filename))
        gender_labels.append(get_gender(file=filename))

    images = np.array(images)
    age_labels = np.array(age_labels)
    gender_labels = np.array(gender_labels)

    model = load_model("model.h5", compile=True)
    predict = model.predict(images)

    for x in range(len(images)):
        print("FOTKA C: ", x)
        print("Age: ", age_labels[x], " Gender: ", gender_labels[x])
        print("Predicted Age: ", np.argmax(predict[0][x]), " Gender: ", np.argmax(predict[1][x]))


guess()

# Záver
V našom projekte sme vytvorili CNN s dvomi výstupnými vrstvami na určenie veku a pohlavie osoby na fotke. Trénovanie a 
úpravy modelu sme experimentálne overovali a snažili sme sa dosahovať lepších výsledkov. Celkovo sme vykonali viac ako
 10 experimentov. Všetky trénovania sú zaznamené v logovacích súboroch. Na zníženie pretrénovania modelu sme použili 
 rôzne regularizátory (l2), taktiež droput a batch-normalization vrstvy. Finálne trénovanie, ktoré bolo monitorované 
 EARLY STOPPINGOM vyšlo na 12 epoch z celkových 30. Stále sme dosahovali pomerne veľkého pretrénovania pri väčších 
 opakovaniach. Ako možné rozšírenia a zlepšenia vidíme v zjednodušení modelu + pridaní regularizátorov na age vetvu a 
 taktiež data augmention pomocou ImageDataGeneratoru. Bohužial ImageDataGenerator sa nám nepodarilo pridať do projektu, 
 aj keď sme sa o to snažili. Celkovo sme sa veľa naučili pri práci na projekte o neurónových sieťach a oboznámili s 
 technikami používanými v praxi.